<a href="https://colab.research.google.com/github/v-demydchenko/Python-for-DA/blob/main/%22HW4_%D0%86%D0%BD%D1%82%D0%B5%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%B2%D0%BD%D1%96_%D0%B2%D1%96%D0%B7%D1%83%D0%B0%D0%BB%D1%96%D0%B7%D0%B0%D1%86%D1%96%D1%97_%D0%B7_Plotly_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнє завдання: Інтерактивні візуалізації з Plotly

## Опис завдання
У цьому домашньому завданні ви будете створювати інтерактивні візуалізації з допомогою бібліотеки Plotly. Ви дізнаєтесь різницю між Plotly Express (швидкі графіки) та Graph Objects (повний контроль), та створите інтерактивний дашборд.

**Опис колонок:**
- `datetime` - дата та час
- `season` - сезон (1=весна, 2=літо, 3=осінь, 4=зима)
- `holiday` - чи є день святковим (0=ні, 1=так)
- `workingday` - чи є день робочим (0=ні, 1=так)
- `weather` - погодні умови (1=ясно, 2=туман, 3=легкий дощ, 4=сильний дощ)
- `temp` - температура в градусах Цельсія
- `atemp` - відчувається як температура
- `humidity` - вологість (%)
- `windspeed` - швидкість вітру
- `casual` - кількість випадкових користувачів
- `registered` - кількість зареєстрованих користувачів
- `count` - загальна кількість орендованих велосипедів

## Підготовка даних


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Завантаження даних
df = pd.read_csv('/content/drive/MyDrive/Data Analyst/data.python/yulu_rental.csv')
df['datetime'] = pd.to_datetime(df['datetime'])

# Для plotly краще не встановлювати datetime як індекс
df['month'] = df['datetime'].dt.month
df['hour'] = df['datetime'].dt.hour
df['weekday'] = df['datetime'].dt.day_name()

# Додаємо назви сезонів
season_map = {1: 'Весна', 2: 'Літо', 3: 'Осінь', 4: 'Зима'}
df['season_name'] = df['season'].map(season_map)

## Завдання 1: Базовий інтерактивний лінійний графік (Plotly Express)

**Завдання:**
Створіть інтерактивний лінійний графік динаміки оренди за часом (рівень деталізації - як в даних) з можливістю zoom та hover.

Дайте відповіді на питання.
**Питання для інтерпретації:**
1. Яка перевага інтерактивного графіка над статичним?
2. Чому на графіку є "пробіли" - ділянки, де одна пряма лінія зʼєднує два "суцільних" блоки з даними? Як би ви це могли дослідити на статичному графіку?


In [4]:
fig = px.line(df, x='datetime', y='count', title='Динаміка оренди велосипедів за часом')
fig.show()
# при наведенні на точку з'являється додаткова інформація. такі графіки більш деталізовані,
# пробіли виникають через відсутність даних

/usr/local/lib/python3.11/dist-packages/kaleido/__init__.py:14: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




## Завдання 2: Scatter plot з додатковими даними (Plotly Express)

**Завдання:**
Створіть scatter plot кількості орендованих велосипедів випадковими користувачами vs кількості орендованих велосипедів зареєстрованими користувачами. Розмір точок встановіть за сумарною кількістю велосипедів, які були взяті в оренду, а колір - за сезоном. В hover_data - додайте деталі, які допоможуть вам в подальшому аналізі.

Дослідіть графік. Зверніть увагу, що ви можете вмикати і вимикати окремі сезони, якщо будете клікати на колір сезону в легенді графіку.

**Дайте відповідь на питання.**
- Як ви проінтерпретуєте роздвоєність цього графіку (дві явні лінії)? Що це означає?
- Які висновки для компанії, які дає велосипеди в оренду, ви можете зробити з цього графіку? 3 основних висновки.

In [5]:
fig = px.scatter(df,
                 x='casual',
                 y='registered',
                 size='count',
                 color='season_name',
                 hover_data=['datetime', 'temp', 'humidity', 'windspeed', 'weather', 'workingday', 'holiday'],
                 title='Співвідношення оренди випадковими та зареєстрованими користувачами за сезонами')
fig.show()

# схоже, що так відображається різниця між робочими та вихідними днями
# 1.послугами компанії користуються багато випадкових користувачів, як правило, у вихідні дні. 2. маркетингові стратегії повинні враховувати видападкових користувачів.

## Завдання 3: Порівняння Plotly Express vs Graph Objects

**Завдання:**
Створіть лінійний графік помісячної динаміки оренди велосипедів двома способами - з Plotly Express та з Graph Objects.

**Дайте відповіді на питання.**
1. Як ви розумієте основну різницю між цими двома підходами?
2. Коли краще використовувати Plotly Express?
3. Коли потрібен Graph Objects?


In [6]:
monthly_counts = df.groupby(df['datetime'].dt.to_period('M'))['count'].sum().reset_index()
monthly_counts['datetime'] = monthly_counts['datetime'].dt.to_timestamp()

fig_px = px.line(monthly_counts,
                 x='datetime',
                 y='count',
                 title='Помісячна динаміка оренди велосипедів Plotly Express')
fig_px.show()

fig_go = go.Figure()

fig_go.add_trace(go.Scatter(x=monthly_counts['datetime'],
                            y=monthly_counts['count'],
                            mode='lines+markers',
                            name='Оренда'))

fig_go.update_layout(title='Помісячна динаміка оренди велосипедів (Graph Objects)',
                     xaxis_title='Дата',
                     yaxis_title='Кількість оренд')

fig_go.show()

# Plotly Express допомагає швидко побудувати, але менш точніше, побудувати потрібний нам графік, оді як Graph Objects дозволяє більш точно, більш "кастомно" побудувати та налаштувати те, щоо нам потрібно
# для швидкої побудови графіків, коли деталізація та точність не потрібні
# коли треба максимальне налаштування для створення чогось складного

## Завдання 4: Дашборд з make_subplots (Graph Objects)

**Завдання:**
Створіть дашборд з 4 різними графіками в одній фігурі:
- Bar chart - середні значення загальної кількості оренд велосипедів за сезонами
- Pie chart - відсоткове співвідношення погодних умов в даних
- Line chart - середнє значення загальної кількості оренд велосипедів за годинами протягом доби
- Scatter plot - кореляція температури vs вологість

Додайте заголовок на дашборд.

**Дайте відповідь на питання**
- На ваш погляд, яка перевага об'єднання графіків в один дашборд?

## Завдання 5: 3D візуалізація

**Завдання:**
Створіть 3D scatter plot для аналізу взаємозв'язку температури, швидкості вітру та загальної кількості орендованих велосипедів. Колір встановіть за сезоном, а розмір - за загальною кількість оренд також.

Дайте відповіді на питання.
**Питання для інтерпретації:**
1. Яку додаткову інформацію, на ваш погляд, дає 3D візуалізація?
2. Чи видно кластери в 3D просторі?
3. Чи ви можете зробити висновки з цієї візуалізації, чи вам було простіше побудувати кілька 2D?



In [7]:
fig = px.scatter_3d(df,
                    x='temp',
                    y='windspeed',
                    z='count',
                    color='season_name',
                    size='count',
                    title='3D Scatter Plot: Температура, Швидкість вітру та Кількість оренд за сезонами')
fig.show()
# длозволяє побачити нам взаємозв'язок між 3-ма змінними, а такоє дає можливість для 3D візуалізації для кращого сприйняття
# думаю, так. наприклад, точки, які відповідають літу та осені (і мають більший розмір)
# як на мене, то більш інформативним є 2D графік. оскільки 3D створює заьагато інформаційного 'шуму'


## Завдання 6: Експорт та збереження інтерактивних графіків

**Завдання:**
Збережіть побудований раніше дашборд в формат HTML. Також змініть вручну щось на дашборді (зум, виділення частини графіку) і збережіть його як статичне зображення через іконку фотоапарату у формат PNG. Завантажте файли з дашбордом у HTML та PNG (або посилання на них на github) разом з посиланням на цей ноутбук при здачі ДЗ.


In [19]:
fig.write_html("dashboard.html")

In [18]:
pip install -U kaleido

In [20]:
!plotly_get_chrome -y

/bin/bash: line 1: plotly_get_chrome: command not found


In [23]:
fig.write_image('graph.png')
# щось виводить помилку. не розумію чому?

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
